In [1]:
"""
permutation (third) step in forward modeling analysis - building a forward model, testing with 2 images held out. This is the submission script to run the analysis, submitting to a SLURM cluster. You MUST edit the last line of this script for your particular submission command. If you do not use this kind of cluster, you should edit the end of the script (where submission occurs) to fit your format.

  Classification framework
  for image1 in all images:
     for image2 in allimages:
         if image1 != image2:
             hold out image 1 and image 2, generate regression parameter matrix using other images
             generate predicted image for image 1 [PR1]
             generate predicted image for image 2 [PR2]
             classify image 1 as fitting best to PR1 or PR2
             classify image 2 as fitting best to PR1 or PR2
"""

import os
import pandas
import sys

from pybraincompare.compare.maths import calculate_correlation
from pybraincompare.compare.mrutils import get_images_df
from pybraincompare.mr.datasets import get_standard_mask
from pybraincompare.mr.transformation import *

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn import linear_model

import nibabel
import numpy
import os
import pandas
import pickle
import sys


from utils import (
   get_base, get_pwd, make_dirs
)

# VARIABLES FOR SLURM
max_runtime="2-00:00"                    # Two days. Each script needs ~10-15 minutes, 30 is recommended for buffer
memory="32000"                           # 16000 might also work
submission_command="sbatch"                  # Your cluster submission command, eg sbatch, qsub
submission_args="-p russpold --qos russpold" # Does not need spaces to left and right


# Get the base and present working directory
base = get_base()
here = get_pwd()

data = os.path.abspath("%s/data" %(here))
results = os.path.abspath("%s/results" %(here))
output_folder = "%s/permutations" %results  

# Make the output directory
make_dirs(output_folder,reason="for permutation results.")

# Images by Concepts data frame
labels_tsv = "%s/concepts_binary_df.tsv" %results
images = pandas.read_csv(labels_tsv,sep="\t",index_col=0)
image_lookup = "%s/image_nii_lookup.pkl" %results

# We will need these folders to exist for job and output files
log_folders = ["%s/.out" %here,"%s/.job" %here]
make_dirs(log_folders)    

# Image metadata with number of subjects included
contrast_file = "%s/filtered_contrast_images.tsv" %results
done = False
for image1_holdout in images.index.tolist():
    print "Parsing %s" %(image1_holdout)
    
    if done:
        break
    
    for image2_holdout in images.index.tolist():
        if (image1_holdout != image2_holdout) and (image1_holdout < image2_holdout):
            output_file = "%s/%s_%s_perform.pkl" %(output_folder,image1_holdout,image2_holdout)
            if not os.path.exists(output_file):

                # Images by Concept data frame, our X
                X = pandas.read_csv(labels_tsv,sep="\t",index_col=0)

                # Images data frame with contrast info, and importantly, number of subjects
                image_df = pandas.read_csv(contrast_file,sep="\t",index_col=0)
                image_df.index = image_df.image_id

                # We should standardize cognitive concepts before modeling
                # http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
                scaled = pandas.DataFrame(StandardScaler().fit_transform(X))
                scaled.columns = X.columns
                scaled.index = X.index
                X = scaled

                # Dictionary to look up image files (4mm)
                lookup = pickle.load(open(image_lookup,"rb"))

                # Get standard mask, 4mm
                standard_mask=get_standard_mask(4)

                # We will save data to dictionary
                result = dict()

                concepts = X.columns.tolist()

                # We will go through each voxel (column) in a data frame of image data
                image_paths = lookup.values()
                mr = get_images_df(file_paths=image_paths,mask=standard_mask)
                image_ids = [int(os.path.basename(x).split(".")[0]) for x in image_paths]
                mr.index = image_ids

                norm = pandas.DataFrame(columns=mr.columns)

                # Normalize the image data by number of subjects
                #V* = V/sqrt(S) 
                for row in mr.iterrows():
                    subid = row[0]
                    number_of_subjects = image_df.loc[subid].number_of_subjects.tolist()
                    norm_vector = row[1]/numpy.sqrt(number_of_subjects)
                    norm.loc[subid] = norm_vector

                del mr

                # Get the labels for holdout images
                holdout1Y = X.loc[image1_holdout,:]
                holdout2Y = X.loc[image2_holdout,:]

                # what we can do is generate a predicted image for a particular set of concepts (e.g, for a left out image) by simply multiplying the concept vector by the regression parameters at each voxel.  then you can do the mitchell trick of asking whether you can accurately classify two left-out images by matching them with the two predicted images. 

                regression_params = pandas.DataFrame(0,index=norm.columns,columns=concepts)
                intercept_params = pandas.DataFrame(0,index=norm.columns,columns=["intercept"])

                print "Training voxels..."
                for voxel in norm.columns:
                    train = [x for x in X.index if x not in [image1_holdout,image2_holdout] and x in norm.index]
                    Y = norm.loc[train,voxel].tolist()
                    Xtrain = X.loc[train,:] 
                    # Use regularized regression
                    clf = linear_model.ElasticNet(alpha=0.1)
                    clf.fit(Xtrain,Y)
                    regression_params.loc[voxel,:] = clf.coef_.tolist()
                    intercept_params.loc[voxel,"intercept"] = clf.intercept_    

                print "Making predictions..."
                # Use regression parameters to generate predicted images
                # data * .coef_ + intercept_
                concept_vector1 =  pandas.DataFrame(holdout1Y)
                concept_vector2 =  pandas.DataFrame(holdout2Y)
                predicted_nii1 =  regression_params.dot(concept_vector1)[image1_holdout] + intercept_params["intercept"] 
                predicted_nii2 =  regression_params.dot(concept_vector2)[image2_holdout] + intercept_params["intercept"]


                # Turn into nifti images
                nii1 = numpy.zeros(standard_mask.shape)
                nii2 = numpy.zeros(standard_mask.shape)
                nii1[standard_mask.get_data()!=0] = predicted_nii1.tolist()
                nii2[standard_mask.get_data()!=0] = predicted_nii2.tolist()
                nii1 = nibabel.Nifti1Image(nii1,affine=standard_mask.get_affine())
                nii2 = nibabel.Nifti1Image(nii2,affine=standard_mask.get_affine())

                # Turn the holdout image data back into nifti
                actual1 = norm.loc[image1_holdout,:]
                actual2 = norm.loc[image2_holdout,:]
                actual_nii1 = numpy.zeros(standard_mask.shape)
                actual_nii2 = numpy.zeros(standard_mask.shape)
                actual_nii1[standard_mask.get_data()!=0] = actual1.tolist()
                actual_nii2[standard_mask.get_data()!=0] = actual2.tolist()
                actual_nii1 = nibabel.Nifti1Image(actual_nii1,affine=standard_mask.get_affine())
                actual_nii2 = nibabel.Nifti1Image(actual_nii2,affine=standard_mask.get_affine())

                # Make a dictionary to lookup images based on nifti
                lookup = dict()
                lookup[actual_nii1] = image1_holdout
                lookup[actual_nii2] = image2_holdout
                lookup[nii1] = image1_holdout
                lookup[nii2] = image2_holdout

                comparison_df = pandas.DataFrame(columns=["actual","predicted","cca_score"])
                comparisons = [[actual_nii1,nii1],[actual_nii1,nii2],[actual_nii2,nii1],[actual_nii2,nii2]]
                count=0
                for comp in comparisons:
                    name1 = lookup[comp[0]]
                    name2 = lookup[comp[1]]
                    corr = calculate_correlation(comp,mask=standard_mask)
                    comparison_df.loc[count] = [name1,name2,corr] 
                    count+=1

                #   actual  predicted  cca_score
                #0    3186       3186   0.908997
                #1    3186        420   0.485644
                #2     420       3186   0.044668
                #3     420        420   0.657109

                result["comparison_df"] = comparison_df

                # Calculate accuracy
                correct = 0
                acc1 = comparison_df[comparison_df.actual==image1_holdout]
                acc2 = comparison_df[comparison_df.actual==image2_holdout]

                # Did we predict image1 to be image1?
                if acc1.loc[acc1.predicted==image1_holdout,"cca_score"].tolist()[0] > acc1.loc[acc1.predicted==image2_holdout,"cca_score"].tolist()[0]:
                    correct+=1

                # Did we predict image2 to be image2?
                if acc2.loc[acc2.predicted==image2_holdout,"cca_score"].tolist()[0] > acc2.loc[acc2.predicted==image1_holdout,"cca_score"].tolist()[0]:
                    correct+=1

                result["number_correct"] = correct

                # We should have a measure of encoding regression performance (in addition to classification). I like out of sample R^2 i.e. how much variance is explained by the model using only cognitive concepts.
                result["r2_%s" %(image1_holdout)] = r2_score(actual1, predicted_nii1)
                result["r2_%s" %(image2_holdout)] = r2_score(actual2, predicted_nii2)

                pickle.dump(result,open(output_file,"wb"))
                print "complete!"
                done = True
                break

                
                
#                 job_id = "%s_%s" %(image1_holdout,image2_holdout)
#                 filey = "%s/.job/class_%s.job" %(here,job_id)
#                 filey = open(filey,"w")
#                 filey.writelines("#!/bin/bash\n")
#                 filey.writelines("#SBATCH --job-name=%s\n" %(job_id))
#                 filey.writelines("#SBATCH --output=%s/.out/%s.out\n" %(here,job_id))
#                 filey.writelines("#SBATCH --error=%s/.out/%s.err\n" %(here,job_id))
#                 filey.writelines("#SBATCH --time=%s\n" %(max_runtime))
#                 filey.writelines("#SBATCH --mem=%s\n" %(memory))
#                 filey.writelines("python %s/2.encoding_regression_performance.py %s %s %s %s %s %s" %(here, 
#                                                                                                       image1_holdout, 
#                                                                                                       image2_holdout, 
#                                                                                                       output_file, 
#                                                                                                       labels_tsv, 
#                                                                                                       image_lookup, 
#                                                                                                       contrast_file))



/home/mjfang/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


BASE project directory is defined as -f
Parsing 111
Training voxels...
Making predictions...


/home/mjfang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:158: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/home/mjfang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:159: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/home/mjfang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:168: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/home/mjfang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:169: DeprecationWarning: get_affine met

complete!
Parsing 3129


In [9]:
predicted_nii1

0        0.011747
1        0.039312
2        0.078127
3        0.014027
4        0.043399
5        0.059229
6       -0.031192
7        0.028993
8        0.041878
9        0.017269
10      -0.036731
11       0.037625
12       0.058208
13       0.024432
14      -0.020017
15      -0.063942
16      -0.048427
17      -0.020679
18      -0.000135
19       0.020998
20      -0.016895
21      -0.054993
22      -0.037133
23      -0.037534
24      -0.035869
25       0.021836
26       0.035510
27       0.024194
28       0.018264
29       0.037066
           ...   
28519    0.000500
28520    0.027448
28521    0.045577
28522    0.017893
28523   -0.002870
28524   -0.024356
28525   -0.037735
28526    0.015849
28527   -0.014313
28528   -0.020697
28529    0.001022
28530   -0.001267
28531   -0.000196
28532   -0.022467
28533   -0.059719
28534   -0.075053
28535   -0.079916
28536   -0.029295
28537    0.009677
28538    0.018777
28539   -0.011627
28540   -0.024022
28541   -0.070966
28542   -0.095829
28543   -0

In [8]:
image1_holdout

3129